## Loading trained model and calculate accuracy on test data

In [1]:
import numpy as np
import MyDL
import MyDL.nn as nn
import os
import fashion_mnist.utils.mnist_reader as mnist_reader


In [2]:
X_test, y_test = mnist_reader.load_mnist('fashion_mnist/data/fashion', kind='t10k')
X_test_mytensor = MyDL.MyTensor(X_test, requires_grad=False)
y_test_mytensor = MyDL.MyTensor(y_test, requires_grad=False)
test_data = MyDL.data.Dataset(X_test_mytensor, y_test_mytensor)

In [3]:
class MLP3(nn.NeuralNetwork):
    def __init__(self, hidden_size1=100, hidden_size2=10, activation='relu'):
        super().__init__()
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.activ_func = activation
        self.fc1 = nn.Linear(784, hidden_size1, initialize='random')
        self.params += self.fc1.params
        self.fc2 = nn.Linear(hidden_size1, hidden_size2, initialize='random')
        self.params += self.fc2.params
        self.fc3 = nn.Linear(hidden_size2, 10, initialize='random')
        self.params += self.fc3.params
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else:
            raise ValueError('Unknown activation function')
        self.softmax = nn.Softmax()
        self.BN1 = nn.BatchNorm1d()
        self.BN2 = nn.BatchNorm1d()
        self.BN3 = nn.BatchNorm1d()
        self.params += self.BN1.params
        self.params += self.BN2.params
        self.params += self.BN3.params
    def forward(self, x):
        x = self.BN1(x)
        x = self.fc1(x)
        x = self.BN2(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.BN3(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

In [4]:
best_model_name = 'MLP3_(100,10)_tanh_L2-0.0_lr-0.1'

In [5]:
model = MLP3(hidden_size1=100, hidden_size2=10, activation='tanh')
model.load(os.path.join('final_model_params', f'{best_model_name}.npz'))
best_model_name = 'MLP3_(100,10)_tanh_L2-0.0_lr-0.1'
result = np.load(os.path.join('final_results', f'{best_model_name}.npz'))
criterion = nn.CrossEntropyLoss()
test_loss, test_acc = MyDL.test(model, test_data, criterion, batch_size = 4 * 256, load_and_eval=True)
print(f"Train Accuracy: {result['train_acc_epoch'][-1]:.3f}")
print(f"Test Accuracy: {test_acc:.3f}")

Train Accuracy: 0.865
Test Accuracy: 0.850
